### Environment Setup and Imports

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time
import threading
import os
from tqdm.auto import tqdm

#TF Logging is excessive, minimize log spamming by setting level lower
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed (only errors)
# 3 = INFO, WARNING, and ERROR messages are not printed

# On Mac you may encounter an error related to OMP, this is a workaround, but slows down the code
# https://github.com/dmlc/xgboost/issues/1715
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

In [ ]:
import tensorflow as tf
tf.config.optimizer.set_jit(True)
AUTOTUNE = tf.data.experimental.AUTOTUNE

## Check if there is any available GPU

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

if gpus:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=15360)]
    )


In [ ]:
from openbot import utils, train

## Hyperparameters

You may have to tune the learning rate and batch size depending on your available compute resources and dataset. As a general rule of thumb, if you increase the batch size by a factor of n, you can increase the learning rate by a factor of sqrt(n). In order to accelerate training and make it more smooth, you should increase the batch size as much as possible. For our working model, we used a batch size of 128. For debugging and hyperparamter tuning, you can set the number of epochs to a small value like 10. If you want to train a model which will achieve good performance, you should set it to 200 or more. For our working model, we used 600.

In [ ]:
params = train.Hyperparameters()
params.MODEL = "resnet"
#Training Parameters
params.TRAIN_BATCH_SIZE = 128
params.TEST_BATCH_SIZE = 128   
params.LEARNING_RATE = 0.0002
params.NUM_EPOCHS = 300    #total number of iterations on the full dataset

#Data Augmentation Parameters
params.BATCH_NORM = False  #normalize the batched data (bad results)
params.TRANS_AUG = True    #translate the batched data (good results)
params.FLIP_AUG = True     #flip the images and steering angles (safe now for turns)
params.USE_LAST = False    #Use last training data

#Callback Parameters
params.WANDB = False       #Use WANDB to log training metrics
params.CHKPT = True        #Save checkpoints at end of every epoch (slows down training)
params.LOG = False         #Save logs
params.TENSORBOARD = False #Use tensorboard to save data

## Load the dataset

In [ ]:
# Now we initiliaze a training object to hold these parameters, set the dataset directories of this object to point to the collected data
tr = train.Training(params, None)

tr.train_data_dir = <train.tfrec>
train.load_tfrecord(tr, verbose=1)

Sanity check on the batch sampling, make sure the images and the steering/throttle look correct! -1 steering is the maximum left turn and 1.0 is the minimum right turn

In [ ]:
#TODO: get rid of the print statements for this
image_batch, label_batch = next(iter(tr.train_ds))
utils.show_train_batch(image_batch.numpy(), label_batch.numpy())

## Training

In [ ]:
# Create broadcast event so we can see the model fit processing
def broadcast(event, payload=None):
    print(event, payload)
    
event = threading.Event()
my_callback = train.MyCallback(broadcast, event)

model, callback_list = train.setup_training(tr, my_callback, "samsung_train_data", verbose=True)

### Begin Training

In [ ]:
tb_callback = tf.keras.callbacks.TensorBoard(log_dir="logdir")

tr.history = model.fit(
    tr.train_ds,
    epochs=tr.hyperparameters.NUM_EPOCHS,
    use_multiprocessing=True,
    validation_data=tr.test_ds,
    verbose=1,
    callbacks=[tb_callback]
)

## Evaluation

The loss and mean absolute error should decrease. This indicates that the model is fitting the data well. The custom metrics (direction and angle) should go towards 1. These provide some additional insight to the training progress. The direction metric measures weather or not predictions are in the same direction as the labels. Similarly the angle metric measures if the prediction is within a small angle of the labels. The intuition is that driving in the right direction with the correct steering angle is most critical part for good final performance.

### Convert to Tflite

In [ ]:
model_name = "resnet.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()
open(model_name, "wb").write(tflite_model)

In [ ]:
import json, os, uuid
path_to_config = "../android/app/src/main/assets/config.json"
model_input = tflite_model.get_signature_list()['serving_default']['inputs'][0]
model_input_name = "serving_default_" + model_input + ":0"
with open(path_to_config, 'r+') as f:
    data = json.load(f)
    data.append({
        'id': data[-1]['id']+1,
        'class': 'BEHAVIORCLONE_F',
        'type': 'BEHAVIORCLONE',
        'name': model_name,
        'pathType': 'ASSET',
        'path': 'networks/' + model_name,
        'inputSize': '224x224',
        'input_name': model_input_name
    })

# create randomly named temporary file to avoid 
# interference with other thread/asynchronous request
tempfile = os.path.join("../android/app/src/main/assets", str(uuid.uuid4()))
with open(tempfile, 'w') as f:
    json.dump(data, f, indent=4)

os.replace(tempfile, path_to_config)


### Loss vs. Validation Loss

In [ ]:
loss = tr.history.history['loss']
val_loss = tr.history.history['val_loss']

In [ ]:
plt.plot(loss, label='loss')
plt.plot(val_loss, label='val_loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="lower right")
plt.savefig("loss.png")

### Mean Absolute Error vs. Validation Mean Absolute Error

In [ ]:
mean_absolute_error = tr.history.history['mean_absolute_error']
val_mean_absolute_error = tr.history.history['val_mean_absolute_error']

In [ ]:
plt.plot(mean_absolute_error, label="mean_absolute_error")
plt.plot(val_mean_absolute_error, label="val_mean_absolute_error")
plt.xlabel("Epoch")
plt.ylabel("Mean Absolute Error")
plt.legend(loc="lower right")
plt.savefig("error.png")

### Direction Metric vs. Validation Direction Metric

In [ ]:
direction_metric = tr.history.history['direction_metric']
val_direction_metric = tr.history.history['val_direction_metric']

In [ ]:
plt.plot(direction_metric, label="direction_metric")
plt.plot(val_direction_metric, label="val_direction_metric")
plt.xlabel("Epoch")
plt.ylabel("Direction Metric")
plt.legend(loc="lower right")
plt.savefig("direction.png")

### Angle Metric vs. Validation Angle Metric

In [ ]:
angle_metric = tr.history.history['angle_metric']
val_angle_metric = tr.history.history['val_angle_metric']

In [ ]:
plt.plot(angle_metric, label="angle_metric")
plt.plot(val_angle_metric, label="val_angle_metric")
plt.xlabel("Epoch")
plt.ylabel("Angle Metric")  
plt.legend(loc="lower right")
plt.savefig("angle.png")

Save tf lite models for best and last checkpoint

In [ ]:
best_index = np.argmax(
     np.array(tr.history.history["val_angle_metric"]))

In [ ]:
best_index

In [ ]:
 #best_index = np.argmax(
 #    np.array(tr.history.history["val_angle_metric"])
 #    + np.array(tr.history.history["val_direction_metric"])
#)
#best_index = np.argmax(np.array(tr.history.history["val_angle_metric"]))
#best_checkpoint = str("cp-%04d.ckpt" % (best_index + 1))
best_checkpoint = "cp-0060.ckpt"
best_tflite = utils.generate_tflite(tr.checkpoint_path, best_checkpoint)
utils.save_tflite(best_tflite, tr.checkpoint_path, "best_")
'''
print(
    "Best Checkpoint (angle_metric: %s, loss: %s, direction_metric: %s): %s"
    % (
        tr.history.history["val_angle_metric"][best_index],
        tr.history.history["val_loss"][best_index],
        tr.history.history["val_mean_absolute_error"][best_index],
        best_checkpoint,
    )
)
'''

In [ ]:
# best_index = np.argmax(
#     np.array(tr.history.history["val_angle_metric"])
#     + np.array(tr.history.history["val_direction_metric"])
# )
best_index = np.argmin(np.array(tr.history.history["val_loss"]))
best_checkpoint = str("cp-%04d.ckpt" % (best_index + 1))
best_tflite = utils.generate_tflite(tr.checkpoint_path, best_checkpoint)
utils.save_tflite(best_tflite, tr.checkpoint_path, "best_loss")
print(
    "Best Checkpoint (angle_metric: %s, loss: %s, direction_metric: %s): %s"
    % (
        tr.history.history["val_angle_metric"][best_index],
        tr.history.history["val_loss"][best_index],
        tr.history.history["val_mean_absolute_error"][best_index],
        best_checkpoint,
    )
)

In [ ]:
last_checkpoint = sorted(
    [
        d
        for d in os.listdir(tr.checkpoint_path)
        if os.path.isdir(os.path.join(tr.checkpoint_path, d))
    ]
)[-1]
last_tflite = utils.generate_tflite(tr.checkpoint_path, last_checkpoint)
utils.save_tflite(last_tflite, tr.checkpoint_path, "last")
print(
    "Last Checkpoint (val_angle: %s, val_direction: %s, val_mean_absolute_error: %s): %s"
    % (
        tr.history.history["val_angle_metric"][-1],
        tr.history.history["val_direction_metric"][-1],
        tr.history.history["val_mean_absolute_error"][-1],
        last_checkpoint,
    )
)

Evaluate the best model

In [ ]:
best_model = utils.load_model(
    os.path.join(tr.checkpoint_path, best_checkpoint),
    tr.loss_fn,
    tr.metric_list,
    tr.custom_objects,
)
test_loss, test_acc, test_dir, test_ang = best_model.evaluate(
    tr.test_ds,
    steps=tr.image_count_test / tr.hyperparameters.TEST_BATCH_SIZE,
    verbose=1,
)

In [ ]:
interpreter = tf.lite.Interpreter("model_tester.tflite")
steering = []
steering2 = []
#interpreter.allocate_tensors()

my_sign = interpreter.get_signature_runner()

In [ ]:
interpreter.get_output_details()

In [ ]:
interpreter.get_input_details()

In [ ]:
best_checkpoint = "cp-0040.ckpt"
model = utils.load_model()

In [ ]:
model = utils.load_model(
    os.path.join(tr.checkpoint_path, "cp-0060.ckpt"),
    tr.loss_fn,
    tr.metric_list,
    tr.custom_objects,
)

In [ ]:

#NUM_SAMPLES = 1

#while NUM_SAMPLES > 0:
image_batch, label_batch = next(iter(tr.test_ds))
#tr.test_ds = tr.test_ds.shuffle(15 * 30)
#max_ind = np.min((NUM_SAMPLES, 128))
pred_batch = model.call(
        tf.slice(image_batch, [0, 0, 0, 0], [15, -1, -1, -1]),
        training=False
)
#interpreter.set_tensor(interpreter.get_input_details()[0]['index'], tf.slice(image_batch, [0, 0, 0, 0], [1, -1, -1, -1]).numpy())
#interpreter.invoke()
#output = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])
#output = my_sign(input_1=tf.slice(image_batch, [0, 0 ,0, 0], [15, -1, -1, -1]))
#NUM_SAMPLES -= 1
#steering.append(pred_batch.numpy())
#steering2.append(output['dense_3'])
#print(NUM_SAMPLES)

utils.show_test_batch(
    image_batch.numpy(), label_batch.numpy(), pred_batch.numpy() #output['dense_3']
)


In [ ]:
#fig,ax = plt.subplots(2,1)
#steering_fixed = np.concatenate(steering, axis=0)[:,1]
steering2_fixed = np.concatenate(steering2, axis=0)[:,1]
#ax[0].hist(steering_fixed, bins=30)
#ax[1].hist(steering2_fixed, bins=20)
plt.hist(steering2_fixed, bins=20)

In [ ]:
utils.compare_tf_tflite(model, tflite_model)

In [ ]:
tr.test_ds = (
        test_dataset.shuffle(tr.hyperparameters.TRAIN_BATCH_SIZE * 10)
        .repeat()
        .batch(tr.hyperparameters.TEST_BATCH_SIZE)
        .prefetch(AUTOTUNE)
    )

In [ ]:
NUM_SAMPLES = 15
image_batch, label_batch = next(iter(tr.test_ds))
pred_batch = best_model.predict(
        tf.slice(image_batch, [0, 0, 0, 0], [NUM_SAMPLES, -1, -1, -1])
)
utils.show_test_batch(
    image_batch.numpy(), label_batch.numpy(), pred_batch
)

In [ ]:
last_checkpoint = sorted(
    [
        d
        for d in os.listdir(tr.checkpoint_path)
        if os.path.isdir(os.path.join(tr.checkpoint_path, d))
    ]
)[-1]
last_tflite = utils.generate_tflite(tr.checkpoint_path, last_checkpoint)
utils.save_tflite(last_tflite, tr.checkpoint_path, "last")

In [ ]:
def process_test_sample(features):
    image = features["image"]
    label = [features["throttle"], features["steer"]]
    return image, label

from openbot import tfrecord_utils
test_dataset = (
        tf.data.TFRecordDataset(tr.test_data_dir, num_parallel_reads=AUTOTUNE)
        .map(tfrecord_utils.parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
        .map(process_test_sample, num_parallel_calls=AUTOTUNE)
    )
tr.test_ds = (
        test_dataset.shuffle(tr.hyperparameters.TRAIN_BATCH_SIZE * 10)
        .repeat()
        .batch(tr.hyperparameters.TEST_BATCH_SIZE)
        .prefetch(AUTOTUNE)
    )

In [ ]:
NUM_SAMPLES = 15
image_batch, label_batch = next(iter(tr.test_ds))
pred_batch = best_model.predict(
        tf.slice(image_batch, [0, 0, 0, 0], [NUM_SAMPLES, -1, -1, -1])
)
utils.show_test_batch(
    image_batch.numpy(), label_batch.numpy(), pred_batch
)

## Save the notebook as HTML

In [ ]:
utils.save_notebook()
current_file = "policy_learning.ipynb"
output_file = os.path.join(tr.log_path, "notebook.html")
utils.output_HTML(current_file, output_file)